<a href="https://colab.research.google.com/github/tercemundo/Chatbot-MIE/blob/main/OllamaColabTunel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configuración Ollama + Streamlit + Cloudflare para Llama 3.2 Chatbot

# 1. Instalamos y configuramos Ollama, Cloudflared y Streamlit
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install streamlit -q

# 2. Primero, vamos a crear el archivo de la aplicación Streamlit usando escritura estándar
import os

# Crear el archivo app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import requests
import json
import os

# Configuraciones de la página
st.set_page_config(
    page_title="Llama 3.2 Chatbot",
    page_icon="🦙",
    layout="centered"
)

# Título de la aplicación
st.title("🦙 Llama 3.2 Chatbot")
st.markdown("Chatbot impulsado por Llama 3.2 (7B) a través de Ollama")

# Inicializar el historial de mensajes
if "messages" not in st.session_state:
    st.session_state.messages = []

# Mostrar mensajes anteriores
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Función para llamar a la API de Ollama
def query_ollama(prompt, history):
    ollama_host = os.environ.get("OLLAMA_HOST", "127.0.0.1")

    # Preparar los mensajes para la API
    messages = []
    for msg in history:
        messages.append({"role": msg["role"], "content": msg["content"]})

    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": prompt})

    # Llamar a la API de Ollama
    response = requests.post(
        f"http://{ollama_host}:11434/api/chat",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model": "llama3.2",
            "messages": messages,
            "stream": False
        })
    )

    if response.status_code == 200:
        result = response.json()
        return result["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"

# Input del usuario
if prompt := st.chat_input("Escribe tu mensaje aquí..."):
    # Añadir el mensaje del usuario al historial
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Mostrar el mensaje del usuario
    with st.chat_message("user"):
        st.markdown(prompt)

    # Mostrar mensaje de espera mientras se procesa
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        message_placeholder.markdown("Pensando...")

        # Obtener la respuesta del modelo
        response = query_ollama(prompt, st.session_state.messages[:-1])

        # Actualizar el placeholder con la respuesta
        message_placeholder.markdown(response)

    # Añadir la respuesta al historial
    st.session_state.messages.append({"role": "assistant", "content": response})
""")

print("✅ Archivo app.py creado correctamente")

# 3. Iniciar manualmente el servidor de Ollama
# Configurar OLLAMA_HOST para permitir conexiones desde cualquier dirección
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

# Iniciar el servidor de Ollama
!nohup ollama serve > ollama.log 2>&1 &
print("⚙️ Iniciando servidor de Ollama en segundo plano...")

# 4. Script para verificar que Ollama está corriendo y descargar el modelo
import time
import socket
import subprocess

def wait_for_service(host, port, max_attempts=60):
    """Espera a que un servicio esté disponible en el puerto especificado."""
    attempts = 0
    while attempts < max_attempts:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex((host, port))
        sock.close()
        if result == 0:
            return True
        time.sleep(1)
        attempts += 1
        if attempts % 5 == 0:
            print(f"⏳ Esperando que el servicio esté disponible ({attempts}/{max_attempts})...")
    return False

# Esperar a que Ollama esté listo
print("⏳ Esperando a que el servidor Ollama esté listo...")
if wait_for_service('127.0.0.1', 11434):
    print("✅ Servidor Ollama iniciado correctamente.")

    # 5. Descargar el modelo
    print("🔄 Descargando modelo Llama 3.2 de 7B (esto puede tardar varios minutos)...")
    try:
        !ollama pull llama3.2
        print("✅ Modelo Llama 3.2 7B descargado correctamente.")
    except:
        print("⚠️ Hubo un problema al descargar el modelo. Verificando disponibilidad...")
        !ollama list
else:
    print("❌ No se pudo iniciar el servidor Ollama. Revisa los logs en ollama.log")

# 6. Iniciar Streamlit y Cloudflare Tunnel
print("🚀 Iniciando aplicación Streamlit...")
!nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

# Esperar a que Streamlit esté listo
print("⏳ Esperando a que Streamlit esté listo...")
if wait_for_service('127.0.0.1', 8501):
    print("✅ Aplicación Streamlit iniciada correctamente.")

    # Iniciar el túnel Cloudflare
    print("🌐 Iniciando túnel Cloudflare para Streamlit...")
    print("🔄 Ejecuta el siguiente comando en una nueva celda para ver la URL del túnel:")
    print("!cloudflared tunnel --url http://127.0.0.1:8501")
else:
    print("❌ No se pudo iniciar Streamlit. Revisa los logs en streamlit.log")

# 7. Añadir comandos para depuración
print("\n== Comandos útiles para depuración ==")
print("Ver logs de Ollama: !cat ollama.log")
print("Ver logs de Streamlit: !cat streamlit.log")
print("Verificar procesos: !ps aux | grep -E 'ollama|streamlit|cloudflared'")
print("Ver modelos disponibles: !ollama list")
print("Detener todos los procesos: !pkill -f 'ollama|streamlit|cloudflared'")
print("Iniciar túnel Cloudflare: !cloudflared tunnel --url http://127.0.0.1:8501")


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Selecting previously unselected package cloudflared.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.4.2) ...
Setting up cloudflared (2025.4.2) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/7

para ver la url publica

ver si ollama corre

In [2]:
!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    27 seconds ago    


In [3]:
!nohup cloudflared tunnel --url http://127.0.0.1:8501 > cloudflared.log 2>&1 &

# Esperar un momento para que se inicie
import time
time.sleep(3)

# Verificar que se está ejecutando
!ps aux | grep cloudflared

# Mostrar la URL del túnel (esto buscará la URL en los logs)
!grep -o "https://.*\.trycloudflare\.com" cloudflared.log || echo "Esperando URL del túnel. Intenta nuevamente en unos segundos con: !grep -o 'https://.*\.trycloudflare\.com' cloudflared.log"

root        1028  7.3  0.2 1259656 32184 ?       Sl   05:20   0:00 cloudflared tunnel --url http://127.0.0.1:8501
root        1045  0.0  0.0   7376  3568 ?        S    05:20   0:00 /bin/bash -c ps aux | grep cloudflared
root        1047  0.0  0.0   6484  2344 ?        S    05:20   0:00 grep cloudflared
Esperando URL del túnel. Intenta nuevamente en unos segundos con: !grep -o 'https://.*\.trycloudflare\.com' cloudflared.log


vamos a ver que modelos tenemos corriendo

In [5]:
!cat cloudflared.log

2025-05-13T05:20:25Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-05-13T05:20:25Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-05-13T05:20:30Z INF +--------------------------------------------------------------------------------------------+
2025-05-13T05:20:30Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-05-13T05:20:30Z INF |  https://pas-stem-sorted-ftp.trycloudflare.com        

In [6]:
!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    54 seconds ago    
